# <div align="center"><b> REPARTIDO-1 </b></div>

## Importaciones

In [38]:
# Standard library imports
import os
import random
from dotenv import load_dotenv

# Third-party library imports
import streamlit as st
import pymupdf

# LangChain imports
from langchain.chains import ConversationChain, LLMChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader

# LangChain Core imports
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import SystemMessage

# Groq-specific imports
from groq import Groq
from langchain_groq import ChatGroq
from pprint import pprint

# Pinecone imports
from pinecone import Pinecone, ServerlessSpec

load_dotenv()



True

## Configuraciones

In [39]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

## Leer el PDF

In [40]:
def read_doc(file_path):
    doc = pymupdf.open(file_path)
    text = ""
    for page in doc:  # iterate the document pages
        text += page.get_text()
    return text

In [41]:
def read_directory(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents = file_loader.load()
    return documents

In [42]:
pdf_directory_path = "./resources/"
pdf_document = read_directory(pdf_directory_path)

In [43]:
pprint(pdf_document)

[Document(metadata={'producer': 'Microsoft® Word para Microsoft 365', 'creator': 'Microsoft® Word para Microsoft 365', 'creationdate': '2025-04-08T18:48:43-03:00', 'author': 'Bruno Masoller', 'moddate': '2025-04-08T18:48:43-03:00', 'source': 'resources\\CV Bruno Masoller - Detallado.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='Página 1 de 4 \n \nCURRICULUM VITAE \nDATOS PERSONALES: \n• NOMBRE: Bruno Martin Masoller Gancedo \n• DIRECCION: Montevideo, Uruguay \n• TELEFONO: +598 91 414 979 \n• MAIL: brunomaso1@hotmail.com | brunomaso1@gmail.com \n• Linkedin: https://www.linkedin.com/in/brunomaso1/ \nFORMACION ACADEMICA: \n• Maestría en Inteligencia Artificial. Universidad de Buenos Aires. Facultad de Ingeniería. Fecha: EN \nCURSO. \n• Especialista en Inteligencia Artificial. Universidad de Buenos Aires. Facultad de Ingeniería. Fecha: \nEN CURSO. \n• Especialista en Ingeniería de Software. Universidad de la República. Facultad de Ingeniería. Fecha: \n2023. \n• Ingen

## Chunking

In [44]:
# Method 1
def chunk_text(text, chunk_size=100):
    chunks = []
    while len(text) > chunk_size:
        chunk = text[:chunk_size]
        text = text[chunk_size:]
        chunks.append(chunk)
    if text:
        chunks.append(text)
    return chunks

In [45]:
# Method 2
def chunk_data(docs, chunk_size=800, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )
    doc = text_splitter.split_documents(docs)
    return doc

In [46]:
# chunks = chunk_text(text=pdf_document, chunk_size=1000)
chunks = chunk_data(docs=pdf_document, chunk_size=100, chunk_overlap=50)
len(chunks)

145

In [47]:
pprint(chunks[0].page_content)

('Página 1 de 4 \n'
 ' \n'
 'CURRICULUM VITAE \n'
 'DATOS PERSONALES: \n'
 '• NOMBRE: Bruno Martin Masoller Gancedo')


## Load documents

In [ ]:
pc = Pinecone(api_key=PINECONE_API_KEY)

In [ ]:
# Create a new index
index_name = "cv-index"
index = pc.create_index(
    name=index_name,
    dimension=1536,
    metric="cosine",
    replicas=1,
    metadata_config={"indexed": ["text"]},
)

TypeError: Pinecone.create_index() got an unexpected keyword argument 'index_name'

In [ ]:
def main():
    """
    Esta función es el punto de entrada principal de la aplicación. Configura el cliente de Groq, la interfaz de Streamlit y maneja la interacción del chat.
    """

    # Obtener la clave API de Groq
    groq_api_key = os.getenv(
        "GROQ_API_KEY"
    )  # Reemplaza 'your_api' con tu clave API real

    # El título y mensaje de bienvenida de la aplicación Streamlit
    st.title("Chat CEIA de ejemplo")
    st.write(
        "¡Hola! Este es un ejemplo de chatbot con memoria persistente gestionada programáticamente con Langchain, utilizando Groq"
    )

    # Agregar opciones de personalización en la barra lateral
    st.sidebar.title("Personalización")
    system_prompt = st.sidebar.text_input("Mensaje del sistema:")
    model = st.sidebar.selectbox(
        "Elige un modelo", ["llama3-8b-8192", "mixtral-8x7b-32768", "gemma-7b-it"]
    )
    conversational_memory_length = st.sidebar.slider(
        "Longitud de la memoria conversacional:", 1, 10, value=5
    )

    memory = ConversationBufferWindowMemory(
        k=conversational_memory_length,
        memory_key="historial_chat",
        return_messages=True,
    )

    user_question = st.text_input("Haz una pregunta:")

    # Variable de estado de la sesión
    if "historial_chat" not in st.session_state:
        st.session_state.historial_chat = []
    else:
        for message in st.session_state.historial_chat:
            memory.save_context({"input": message["humano"]}, {"output": message["IA"]})

    # Inicializar el objeto de chat Groq con Langchain
    groq_chat = ChatGroq(groq_api_key=groq_api_key, model_name=model)

    # Si el usuario ha hecho una pregunta,
    if user_question:

        # Construir una plantilla de mensaje de chat utilizando varios componentes
        prompt = ChatPromptTemplate.from_messages(
            [
                SystemMessage(
                    content=system_prompt
                ),  # Este es el mensaje del sistema persistente que siempre se incluye al inicio del chat.
                MessagesPlaceholder(
                    variable_name="historial_chat"
                ),  # Este marcador de posición será reemplazado por el historial de chat real durante la conversación. Ayuda a mantener el contexto.
                HumanMessagePromptTemplate.from_template(
                    "{human_input}"
                ),  # Esta plantilla es donde se inyectará la entrada actual del usuario en el mensaje.
            ]
        )

        # Crear una cadena de conversación utilizando el LLM (Modelo de Lenguaje) de LangChain
        conversation = LLMChain(
            llm=groq_chat,  # El objeto de chat Groq LangChain inicializado anteriormente.
            prompt=prompt,  # La plantilla de mensaje construida.
            verbose=True,  # Habilita la salida detallada, lo cual puede ser útil para depurar.
            memory=memory,  # El objeto de memoria conversacional que almacena y gestiona el historial de la conversación.
        )

        # La respuesta del chatbot se genera enviando el mensaje completo a la API de Groq.
        response = conversation.predict(human_input=user_question)
        message = {"humano": user_question, "IA": response}
        st.session_state.historial_chat.append(message)
        st.write("Chatbot:", response)


if __name__ == "__main__":
    main()